In [2]:
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import numpy as np
import random
import pandas as pd
pd.set_option('float_format', '{:f}'.format)
pd.set_option('display.max_columns', None)
from matplotlib.pyplot import figure
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from math import floor
import os
import pickle
from sklearn.preprocessing import MinMaxScaler
import time

In [3]:
cat_list=[ 'sensor','location','location2','message','senor_type','lastSensorEventHours','lastSensorDayOfWeek','prevDominantSensor1','prevDominantSensor2','lastSensorID','lastSensorLocation','lastMotionLocation']
num_list=[ 'lastSensorEventSeconds','windowDuration','timeSinceLastSensorEvent','complexity','activityChange','areaTransitions','sensorCount-Bathroom','sensorCount-Bedroom','sensorCount-Chair','sensorCount-DiningRoom','sensorCount-Hall','sensorCount-Ignore','sensorCount-Kitchen','sensorCount-LivingRoom','sensorCount-Office','sensorCount-OutsideDoor','sensorCount-WorkArea','sensorElTime-Bathroom','sensorElTime-Bedroom','sensorElTime-Chair','sensorElTime-DiningRoom','sensorElTime-Hall','sensorElTime-Ignore','sensorElTime-Kitchen','sensorElTime-LivingRoom','sensorElTime-Office','sensorElTime-OutsideDoor','sensorElTime-WorkArea']

### Reading the combined dataset

In [4]:
i=0
X_train=pd.read_csv(r'model_dataset\csh_X_train.csv')
X_train=X_train.drop(['numDistinctSensors'],axis=1)
X_train[cat_list]=X_train[cat_list].astype('str')

y_train=pd.read_csv(r'model_dataset\csh_y_train.csv')

X_train

,sensor,location,location2,message,senor_type,lastSensorEventHours,lastSensorDayOfWeek,prevDominantSensor1,prevDominantSensor2,lastSensorID,lastSensorLocation,lastMotionLocation,lastSensorEventSeconds,windowDuration,timeSinceLastSensorEvent,complexity,activityChange,areaTransitions,sensorCount-Bathroom,sensorCount-Bedroom,sensorCount-Chair,sensorCount-DiningRoom,sensorCount-Hall,sensorCount-Ignore,sensorCount-Kitchen,sensorCount-LivingRoom,sensorCount-Office,sensorCount-OutsideDoor,sensorCount-WorkArea,sensorElTime-Bathroom,sensorElTime-Bedroom,sensorElTime-Chair,sensorElTime-DiningRoom,sensorElTime-Hall,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-Office,sensorElTime-OutsideDoor,sensorElTime-WorkArea
0,M008,Kitchen,Kitchen,OFF,Control4-Motion,18.0,4.0,6.0,5.0,6.0,6.0,6.0,67236.000000,59.000000,0.000000,0.783777,0.389831,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.170000,26.180000,0.000000,0.000000,0.000000,0.000000,4631.252905,4195.487419,86400.000000,118.449243,86400.000000,0.402761,0.000000,119.516528,86400.000000,3770.352859,4189.263011
1,MA013,LivingRoom,LivingRoom,ON,Control4-MotionArea,16.0,3.0,0.0,5.0,7.0,7.0,7.0,58646.000000,61.000000,0.000000,1.403674,0.573770,1.000000,20.090000,0.000000,0.000000,0.000000,0.000000,9.150000,0.000000,5.110000,0.000000,0.000000,0.000000,13.478419,17156.081468,86400.000000,19501.182607,86400.000000,8.475715,19425.574446,0.000000,86400.000000,265.409013,86400.000000
2,M008,LivingRoom,Chair,ON,Control4-Motion,9.0,1.0,6.0,6.0,7.0,7.0,7.0,34958.000000,320.000000,8.000000,1.241946,0.181250,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.300000,0.000000,22.240000,0.000000,4.810000,0.000000,3784.110391,3939.652611,86400.000000,350.534183,86400.000000,265.942078,350.704024,0.000000,86400.000000,308.349595,86400.000000
3,LS022,Ignore,Ignore,36,Control4-LightSensor,14.0,5.0,6.0,6.0,5.0,5.0,6.0,51459.000000,37.000000,1.000000,0.918296,0.594595,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,11.430000,22.920000,0.000000,0.000000,0.000000,0.000000,4332.595631,456.210724,1743.778791,86400.000000,86400.000000,0.000000,1.003861,437.759688,86400.000000,4441.825389,1750.291579
4,LS020,Ignore,Ignore,9,Control4-LightSensor,12.0,3.0,6.0,5.0,5.0,5.0,6.0,45166.000000,69.000000,2.000000,0.468996,0.579710,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.300000,31.050000,0.000000,0.000000,0.000000,0.000000,1521.735071,6386.382299,86400.000000,181.398060,86400.000000,0.000000,2.179429,196.038538,86400.000000,1980.381580,1454.150398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5901212,LS009,Ignore,Ignore,05,Control4-LightSensor,16.0,0.0,5.0,7.0,5.0,5.0,7.0,58266.000000,2227.000000,12.000000,0.996792,0.710822,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.100000,0.000000,16.250000,0.000000,0.000000,0.000000,5643.498956,3756.011759,86400.000000,86400.000000,86400.000000,0.000000,3635.045358,12.423270,86400.000000,3764.466846,86400.000000
5901213,MA010,Bedroom,Bedroom,ON,Control4-MotionArea,22.0,5.0,1.0,1.0,1.0,1.0,1.0,82489.000000,556.000000,1.000000,1.038921,0.050360,6.000000,0.000000,27.570000,0.000000,0.000000,0.000000,2.090000,0.000000,2.430000,0.000000,2.260000,0.000000,750.874976,0.000000,86400.000000,86400.000000,86400.000000,374.295954,1568.456759,549.309264,86400.000000,527.154053,86400.000000
5901214,MA009,LivingRoom,Chair,OFF,Control4-MotionArea,8.0,1.0,1.0,1.0,7.0,7.0,7.0,31195.000000,21.000000,0.000000,2.050195,0.714286,6.000000,1.250000,6.880000,0.000000,0.000000,0.000000,13.940000,3.090000,2.100000,0.000000,7.090000,0.000000,19.564023,2.373453,86400.000000,86400.000000,86400.000000,0.856103,0.075646,0.000000,86400.000000,4.727912,86400.000000
5901215,LS011,Ignore,Ignore,40,Control4-LightSensor,15.0,2.0,6.0,6.0,5.0,5.0,1.0,57060.000000,38.000000,0.000000,1.570951,0.236842,7.000000,0.000000,10.690000,0.000000,0.0

In [6]:
X_train[cat_list].iloc[:10]

,sensor,location,location2,message,senor_type,lastSensorEventHours,lastSensorDayOfWeek,prevDominantSensor1,prevDominantSensor2,lastSensorID,lastSensorLocation,lastMotionLocation
0,M008,Kitchen,Kitchen,OFF,Control4-Motion,18.0,4.0,6.0,5.0,6.0,6.0,6.0
1,MA013,LivingRoom,LivingRoom,ON,Control4-MotionArea,16.0,3.0,0.0,5.0,7.0,7.0,7.0
2,M008,LivingRoom,Chair,ON,Control4-Motion,9.0,1.0,6.0,6.0,7.0,7.0,7.0
3,LS022,Ignore,Ignore,36,Control4-LightSensor,14.0,5.0,6.0,6.0,5.0,5.0,6.0
4,LS020,Ignore,Ignore,9,Control4-LightSensor,12.0,3.0,6.0,5.0,5.0,5.0,6.0
5,M004,LivingRoom,Chair,OFF,Control4-Motion,20.0,1.0,5.0,3.0,7.0,7.0,7.0
6,LS017,Ignore,Ignore,28,Control4-LightSensor,16.0,0.0,5.0,6.0,5.0,5.0,6.0
7,LS002,Ignore,Ignore,5,Control4-LightSensor,18.0,0.0,6.0,6.0,5.0,5.0,3.0
8,MA015,Bedroom,Bedroom,ON,Control4-MotionArea,16.0,3.0,5.0,10.0,1.0,1.0,1.0
9,M008,Kitchen,Kitchen,ON,Control4-Motion,13.0,3.0,6.0,6.0,6.0,6.0,6.0


### Percentile distributioin of numeric variables

In [4]:
percentile_dist=X_train.describe(percentiles=[0.001,0.01,0.025,0.05,.1,.2,.3,.4,.5,.6,.7,.8,.9,.95,.975,.999,.9995])
percentile_dist

,lastSensorEventSeconds,windowDuration,timeSinceLastSensorEvent,complexity,activityChange,areaTransitions,sensorCount-Bathroom,sensorCount-Bedroom,sensorCount-Chair,sensorCount-DiningRoom,sensorCount-Hall,sensorCount-Ignore,sensorCount-Kitchen,sensorCount-LivingRoom,sensorCount-Office,sensorCount-OutsideDoor,sensorCount-WorkArea,sensorElTime-Bathroom,sensorElTime-Bedroom,sensorElTime-Chair,sensorElTime-DiningRoom,sensorElTime-Hall,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-Office,sensorElTime-OutsideDoor,sensorElTime-WorkArea
count,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000
mean,48721.208091,418.040915,14.930986,0.977976,0.512131,1.678086,4.698522,5.790877,0.276697,0.725776,0.047359,9.345105,7.349678,3.530599,0.095005,0.632621,1.857760,3581.883102,2835.284154,59081.535855,37114.358862,83169.021885,2603.670968,4332.349658,7860.426005,83577.772945,7149.817161,33830.468495
std,18858.549418,1382.782549,148.175475,0.581771,0.213151,3.026762,9.324243,10.246733,1.774655,3.110288,0.489095,7.688312,11.198325,7.062904,1.357323,2.406117,6.107165,9355.279082,7688.437945,37900.715762,40520.375696,16231.331186,14420.183684,10394.543853,16230.172968,14757.204920,12906.481617,38808.342147
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.1%,267.000000,10.000000,0.000000,0.000000,0.004580,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1%,2717.000000,16.000000,0.000000,0.000000,0.027682,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.701422,0.000000,41.740800,0.000000,0.000000,0.000000,305.969530,0.000000,0.000000
2.5%,9694.000000,20.000000,0.000000,0.000000,0.065844,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.511642,0.615125,604.061093,0.000000,0.000000,0.000000,5962.417752,0.561588,0.000000
5%,22312.000000,25.000000,0.000000,0.000000,0.129630,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,106.791818,8.464174,86400.000000,0.000000,0.000000,0.000000,86400.000000,10.025165,0.000000
10%,27205.000000,31.000000,0.000000,0.047279,0.224299,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.190000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,459.945302,52.607452,86400.000000,0.000000,0.000000,0.004875,86400.000000,66.077964,24.930908
20%,31632.000000,43.000000,0.000000,0.511117,0.334385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.460000,0.000000,0.000000,0.000000,0.000000,0.000000,5.859971,1.640805,2726.351976,282.230484,86400.000000,0.000000,0.000000,15.559223,86400.000000,279.528380,336.820471


### Treating Outliers

In [5]:
outlier_treat=['windowDuration','timeSinceLastSensorEvent','activityChange','areaTransitions']

# limiters=pd.concat([percentile_dist.loc['97.5%',['windowDuration','timeSinceLastSensorEvent']],percentile_dist.loc['99.95%',['activityChange']]])
# limiters.to_csv(r'model_dataset\outlier_thresholds.csv')
limiters=pd.read_csv(r'model_dataset\outlier_thresholds.csv',index_col=0)
for column in outlier_treat:
    X_train.loc[(X_train[column]>=limiters.loc[column,'0']),column]=limiters.loc[column,'0']
limiters

,0
windowDuration,1000
timeSinceLastSensorEvent,100
activityChange,1
areaTransitions,10


In [6]:
percentile_dist2=X_train.describe(percentiles=[0.001,0.01,0.025,0.05,.1,.2,.3,.4,.5,.6,.7,.8,.9,.95,.975,.999,.9995])
percentile_dist2

,lastSensorEventSeconds,windowDuration,timeSinceLastSensorEvent,complexity,activityChange,areaTransitions,sensorCount-Bathroom,sensorCount-Bedroom,sensorCount-Chair,sensorCount-DiningRoom,sensorCount-Hall,sensorCount-Ignore,sensorCount-Kitchen,sensorCount-LivingRoom,sensorCount-Office,sensorCount-OutsideDoor,sensorCount-WorkArea,sensorElTime-Bathroom,sensorElTime-Bedroom,sensorElTime-Chair,sensorElTime-DiningRoom,sensorElTime-Hall,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-Office,sensorElTime-OutsideDoor,sensorElTime-WorkArea
count,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000
mean,48721.208091,228.843790,7.278982,0.977976,0.512127,1.593792,4.698522,5.790877,0.276697,0.725776,0.047359,9.345105,7.349678,3.530599,0.095005,0.632621,1.857760,3581.883102,2835.284154,59081.535855,37114.358862,83169.021885,2603.670968,4332.349658,7860.426005,83577.772945,7149.817161,33830.468495
std,18858.549418,304.031858,19.280523,0.581771,0.213093,2.711435,9.324243,10.246733,1.774655,3.110288,0.489095,7.688312,11.198325,7.062904,1.357323,2.406117,6.107165,9355.279082,7688.437945,37900.715762,40520.375696,16231.331186,14420.183684,10394.543853,16230.172968,14757.204920,12906.481617,38808.342147
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.1%,267.000000,10.000000,0.000000,0.000000,0.004580,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1%,2717.000000,16.000000,0.000000,0.000000,0.027682,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.701422,0.000000,41.740800,0.000000,0.000000,0.000000,305.969530,0.000000,0.000000
2.5%,9694.000000,20.000000,0.000000,0.000000,0.065844,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,17.511642,0.615125,604.061093,0.000000,0.000000,0.000000,5962.417752,0.561588,0.000000
5%,22312.000000,25.000000,0.000000,0.000000,0.129630,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,106.791818,8.464174,86400.000000,0.000000,0.000000,0.000000,86400.000000,10.025165,0.000000
10%,27205.000000,31.000000,0.000000,0.047279,0.224299,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.190000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,459.945302,52.607452,86400.000000,0.000000,0.000000,0.004875,86400.000000,66.077964,24.930908
20%,31632.000000,43.000000,0.000000,0.511117,0.334385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.460000,0.000000,0.000000,0.000000,0.000000,0.000000,5.859971,1.640805,2726.351976,282.230484,86400.000000,0.000000,0.000000,15.559223,86400.000000,279.528380,336.820471


### MinMaxScaling the numeric variables

In [7]:
# scaler = MinMaxScaler()
# scaler.fit(X_train[num_list])
# file = open('min_max_scaler.pkl', 'wb')
# pickle.dump(scaler, file)
# file.close()
file = open('min_max_scaler.pkl', 'rb')
scaler = pickle.load(file)
file.close()
X_train[num_list]=scaler.transform(X_train[num_list])

In [8]:
percentile_dist3=X_train.describe(percentiles=[0.001,0.01,0.025,0.05,.1,.2,.3,.4,.5,.6,.7,.8,.9,.95,.975,.999,.9995])
percentile_dist3

,lastSensorEventSeconds,windowDuration,timeSinceLastSensorEvent,complexity,activityChange,areaTransitions,sensorCount-Bathroom,sensorCount-Bedroom,sensorCount-Chair,sensorCount-DiningRoom,sensorCount-Hall,sensorCount-Ignore,sensorCount-Kitchen,sensorCount-LivingRoom,sensorCount-Office,sensorCount-OutsideDoor,sensorCount-WorkArea,sensorElTime-Bathroom,sensorElTime-Bedroom,sensorElTime-Chair,sensorElTime-DiningRoom,sensorElTime-Hall,sensorElTime-Ignore,sensorElTime-Kitchen,sensorElTime-LivingRoom,sensorElTime-Office,sensorElTime-OutsideDoor,sensorElTime-WorkArea
count,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000,5901217.000000
mean,0.563909,0.228072,0.072790,0.318949,0.512127,0.159379,0.136784,0.168584,0.008055,0.021129,0.001422,0.272055,0.213964,0.102783,0.002766,0.018417,0.054083,0.041457,0.032816,0.683814,0.429564,0.962604,0.030135,0.050143,0.090977,0.967335,0.082753,0.391556
std,0.218273,0.304336,0.192805,0.189734,0.213093,0.271143,0.271448,0.298304,0.051664,0.090547,0.014688,0.223823,0.326007,0.205616,0.039514,0.070047,0.177792,0.108279,0.088987,0.438666,0.468986,0.187863,0.166900,0.120307,0.187849,0.170801,0.149381,0.449171
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
0.1%,0.003090,0.009009,0.000000,0.000000,0.004580,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1%,0.031447,0.015015,0.000000,0.000000,0.027682,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000008,0.000000,0.000483,0.000000,0.000000,0.000000,0.003541,0.000000,0.000000
2.5%,0.112200,0.019019,0.000000,0.000000,0.065844,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000203,0.000007,0.006991,0.000000,0.000000,0.000000,0.069009,0.000006,0.000000
5%,0.258244,0.024024,0.000000,0.000000,0.129630,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001236,0.000098,1.000000,0.000000,0.000000,0.000000,1.000000,0.000116,0.000000
10%,0.314876,0.030030,0.000000,0.015419,0.224299,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.034643,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005323,0.000609,1.000000,0.000000,0.000000,0.000000,1.000000,0.000765,0.000289
20%,0.366115,0.042042,0.000000,0.166691,0.334385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100728,0.000000,0.000000,0.000000,0.000000,0.000000,0.000068,0.000019,0.031555,0.003267,1.000000,0.000000,0.000000,0.000180,1.000000,0.003235,0.003898


### Grouping of Light sensor messages

In [9]:
lightsens=pd.DataFrame(X_train.loc[(X_train['senor_type']=='Control4-LightSensor')|(X_train['senor_type']=='Control4-Light'),'message']).astype('int')
lightsens.describe(percentiles=[0.001,0.01,0.025,0.05,.1,.2,.3,.33,.4,.5,.6,.67,.7,.8,.9,.95,.975,.999,.9995])

,message
count,1537086.000000
mean,22.416496
std,18.810586
min,0.000000
0.1%,0.000000
1%,0.000000
2.5%,0.000000
5%,1.000000
10%,3.000000
20%,6.000000


### Grouping of Temparature sensor messages

In [10]:
lightsens.loc[lightsens['message']==0,'group']='0'
lightsens.loc[(lightsens['message']>0)&(lightsens['message']<=3),'group']='1-3'
lightsens.loc[(lightsens['message']>3)&(lightsens['message']<=6),'group']='4-6'
lightsens.loc[(lightsens['message']>6)&(lightsens['message']<=10),'group']='7-10'
lightsens.loc[(lightsens['message']>10)&(lightsens['message']<=13),'group']='11-13'
lightsens.loc[(lightsens['message']>13)&(lightsens['message']<=18),'group']='14-18'
lightsens.loc[(lightsens['message']>18)&(lightsens['message']<=22),'group']='19-22'
lightsens.loc[(lightsens['message']>22)&(lightsens['message']<=28),'group']='23-28'
lightsens.loc[(lightsens['message']>28)&(lightsens['message']<=38),'group']='29-38'
lightsens.loc[(lightsens['message']>38)&(lightsens['message']<=50),'group']='39-50'
lightsens.loc[(lightsens['message']>50),'group']='>50'

X_train.loc[(X_train['senor_type']=='Control4-LightSensor')|(X_train['senor_type']=='Control4-Light'),'message']=lightsens['group']
X_train.loc[(X_train['senor_type']=='Control4-LightSensor')|(X_train['senor_type']=='Control4-Light')]['message'].value_counts()

14-18    186241
7-10     180613
29-38    165165
39-50    147734
23-28    147403
>50      146887
1-3      140126
4-6      131799
19-22    125422
11-13    112768
0         52928
Name: message, dtype: int64

In [11]:
tempsens=pd.DataFrame(X_train[(X_train['senor_type']=='Control4-Temperature')]['message'].astype('int'))
tempsens.describe(percentiles=[0.001,0.01,0.025,0.05,.1,.2,.3,.33,.4,.5,.6,.67,.7,.8,.9,.95,.975,.999,.9995])

,message
count,56037.000000
mean,26.561522
std,4.441322
min,9.000000
0.1%,10.000000
1%,20.000000
2.5%,21.000000
5%,22.000000
10%,23.000000
20%,24.000000


In [12]:
tempsens.loc[tempsens['message']<=24,'group']='Cold'
tempsens.loc[tempsens['message']>24,'group']='Medium'
tempsens.loc[tempsens['message']>=27,'group']='Hot'
X_train.loc[(X_train['senor_type']=='Control4-Temperature'),'message']=tempsens['group']
X_train[(X_train['senor_type']=='Control4-Temperature')]['message']

259           Hot
664           Hot
678          Cold
694        Medium
698          Cold
            ...  
5900693      Cold
5900744      Cold
5901084    Medium
5901159    Medium
5901201    Medium
Name: message, Length: 56037, dtype: object

In [13]:
weekday_map={'0.0':'0_Monday','1.0':'1_Tuesday','2.0':'2_Wednesday','3.0':'3_Thursday','4.0':'4_Friday','5.0':'5_Saturday','6.0':'6_Sunday'}
X_train['lastSensorDayOfWeek']=X_train['lastSensorDayOfWeek'].apply(lambda x: weekday_map[x])

In [14]:
X_train.to_csv(r'model_dataset\csh_X_train_preprocessed.csv',index=None)

### Running the above code on loop

In [17]:
X_test=pd.read_csv(r'model_dataset\csh_X_test.csv')
X_test=X_test.drop(['numDistinctSensors'],axis=1)
X_test[cat_list]=X_test[cat_list].astype('str')

outlier_treat=['windowDuration','timeSinceLastSensorEvent','activityChange','areaTransitions']
limiters=pd.read_csv(r'model_dataset\outlier_thresholds.csv',index_col=0)
for column in outlier_treat:
    X_test.loc[(X_test[column]>=limiters.loc[column,'0']),column]=limiters.loc[column,'0']

file = open('min_max_scaler.pkl', 'rb')
scaler = pickle.load(file)
file.close()
X_test[num_list]=scaler.transform(X_test[num_list])

lightsens=pd.DataFrame(X_test.loc[(X_test['senor_type']=='Control4-LightSensor')|(X_test['senor_type']=='Control4-Light'),'message']).astype('int')
lightsens.loc[lightsens['message']==0,'group']='0'
lightsens.loc[(lightsens['message']>0)&(lightsens['message']<=3),'group']='1-3'
lightsens.loc[(lightsens['message']>3)&(lightsens['message']<=6),'group']='4-6'
lightsens.loc[(lightsens['message']>6)&(lightsens['message']<=10),'group']='7-10'
lightsens.loc[(lightsens['message']>10)&(lightsens['message']<=13),'group']='11-13'
lightsens.loc[(lightsens['message']>13)&(lightsens['message']<=18),'group']='14-18'
lightsens.loc[(lightsens['message']>18)&(lightsens['message']<=22),'group']='19-22'
lightsens.loc[(lightsens['message']>22)&(lightsens['message']<=28),'group']='23-28'
lightsens.loc[(lightsens['message']>28)&(lightsens['message']<=38),'group']='29-38'
lightsens.loc[(lightsens['message']>38)&(lightsens['message']<=50),'group']='39-50'
lightsens.loc[(lightsens['message']>50),'group']='>50'
X_test.loc[(X_test['senor_type']=='Control4-LightSensor')|(X_test['senor_type']=='Control4-Light'),'message']=lightsens['group']
X_test.loc[(X_test['senor_type']=='Control4-LightSensor')|(X_test['senor_type']=='Control4-Light')]['message'].value_counts()

tempsens=pd.DataFrame(X_test[(X_test['senor_type']=='Control4-Temperature')]['message'].astype('int'))
tempsens.loc[tempsens['message']<=24,'group']='Cold'
tempsens.loc[tempsens['message']>24,'group']='Medium'
tempsens.loc[tempsens['message']>=27,'group']='Hot'
X_test.loc[(X_test['senor_type']=='Control4-Temperature'),'message']=tempsens['group']

weekday_map={'0.0':'0_Monday','1.0':'1_Tuesday','2.0':'2_Wednesday','3.0':'3_Thursday','4.0':'4_Friday','5.0':'5_Saturday','6.0':'6_Sunday'}
X_test['lastSensorDayOfWeek']=X_test['lastSensorDayOfWeek'].apply(lambda x: weekday_map[x])

X_test.to_csv(r'model_dataset\csh_X_test_preprocessed.csv',index=None)

X_valid=pd.read_csv(r'model_dataset\csh_X_valid.csv')
X_valid=X_valid.drop(['numDistinctSensors'],axis=1)
X_valid[cat_list]=X_valid[cat_list].astype('str')

outlier_treat=['windowDuration','timeSinceLastSensorEvent','activityChange','areaTransitions']
limiters=pd.read_csv(r'model_dataset\outlier_thresholds.csv',index_col=0)
for column in outlier_treat:
    X_valid.loc[(X_valid[column]>=limiters.loc[column,'0']),column]=limiters.loc[column,'0']

file = open('min_max_scaler.pkl', 'rb')
scaler = pickle.load(file)
file.close()
X_valid[num_list]=scaler.transform(X_valid[num_list])    

lightsens=pd.DataFrame(X_valid.loc[(X_valid['senor_type']=='Control4-LightSensor')|(X_valid['senor_type']=='Control4-Light'),'message']).astype('int')
lightsens.loc[lightsens['message']==0,'group']='0'
lightsens.loc[(lightsens['message']>0)&(lightsens['message']<=3),'group']='1-3'
lightsens.loc[(lightsens['message']>3)&(lightsens['message']<=6),'group']='4-6'
lightsens.loc[(lightsens['message']>6)&(lightsens['message']<=10),'group']='7-10'
lightsens.loc[(lightsens['message']>10)&(lightsens['message']<=13),'group']='11-13'
lightsens.loc[(lightsens['message']>13)&(lightsens['message']<=18),'group']='14-18'
lightsens.loc[(lightsens['message']>18)&(lightsens['message']<=22),'group']='19-22'
lightsens.loc[(lightsens['message']>22)&(lightsens['message']<=28),'group']='23-28'
lightsens.loc[(lightsens['message']>28)&(lightsens['message']<=38),'group']='29-38'
lightsens.loc[(lightsens['message']>38)&(lightsens['message']<=50),'group']='39-50'
lightsens.loc[(lightsens['message']>50),'group']='>50'
X_valid.loc[(X_valid['senor_type']=='Control4-LightSensor')|(X_valid['senor_type']=='Control4-Light'),'message']=lightsens['group']
X_valid.loc[(X_valid['senor_type']=='Control4-LightSensor')|(X_valid['senor_type']=='Control4-Light')]['message'].value_counts()

tempsens=pd.DataFrame(X_valid[(X_valid['senor_type']=='Control4-Temperature')]['message'].astype('int'))
tempsens.describe(percentiles=[0.001,0.01,0.025,0.05,.1,.2,.3,.33,.4,.5,.6,.67,.7,.8,.9,.95,.975,.999,.9995])

tempsens.loc[tempsens['message']<=24,'group']='Cold'
tempsens.loc[tempsens['message']>24,'group']='Medium'
tempsens.loc[tempsens['message']>=27,'group']='Hot'
X_valid.loc[(X_valid['senor_type']=='Control4-Temperature'),'message']=tempsens['group']

weekday_map={'0.0':'0_Monday','1.0':'1_Tuesday','2.0':'2_Wednesday','3.0':'3_Thursday','4.0':'4_Friday','5.0':'5_Saturday','6.0':'6_Sunday'}
X_valid['lastSensorDayOfWeek']=X_valid['lastSensorDayOfWeek'].apply(lambda x: weekday_map[x])

X_valid.to_csv(r'model_dataset\csh_X_valid_preprocessed.csv',index=None)